<a href="https://colab.research.google.com/github/JeanMusenga/TOSEM-2025-Submission/blob/main/GPT_4o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import all necessary modules or libraries

In [ ]:
import openai
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from openai import OpenAI
import os
from IPython.display import Markdown

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [ ]:
# Initialize OpenAI client
client = OpenAI(api_key="xxxxx")
 # This is a free api key

## Load dataset

In [ ]:
# Load dataset
dataset = pd.read_excel("200_ARPs_and_Programming_Posts.xlsx")

# Define a function to classify text using GPT-4o

In [ ]:
# Create a valiable to store results
result_gpt4 = []

# Define a function to classify text using GPT-4o with retry logic by process each row
def gpt4_classify(text, max_retries=5):
    delay = 1  # Initialize delay before the loop
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create( # Use `client` instead of `openai.Chat.Completion.create`
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are a classifier that determines whether a text is related to software architecture design. Reply with only “1” (architecture) or “0” (not architecture)."},
                    {"role": "user", "content": f"Tell me if the following text is related to an architectural design discussion or software programming discussion. Just say 1 for  architectural design discussion or 0 for programming discussion.\n\n{text}"}
                ],
                max_tokens=5,
                temperature=0,
            )
            return response.choices[0].message.content.strip() #Correct way to access response. New OpenAI Python API (openai>=1.0.0) uses object attributes instead of dictionaries
                                                           #(e.g., response["choices"][0]["message"]["content"] ❌ is incorrect in the new version).

        except openai.OpenAIError as e:  # Catch OpenAI API errors
            print(f"Error: {e}. Retrying in {delay} seconds...")
            time.sleep(delay)
            delay *= 2  # Exponential backoff (1s → 2s → 4s → 8s → ...)

    return None  # Return None if all retries fail


# Process each row
This process is very solow. It iterate each row in the dataset and that is it execute each row per second not a batch of posts and takes too much to finish

In [ ]:
# variable to store predicted results
result_gpt4 = []
# Process each row
for index, row in tqdm(dataset.iterrows(), total=dataset.shape[0]):
    gpt_response = gpt4_classify(row['Question_body'])

    if gpt_response is None:
        result_gpt4.append("Error")
    else:
        if "Yes" in gpt_response:
            result_gpt4.append(1)
        elif "No" in gpt_response:
            result_gpt4.append(0)
        else:
            print(f"Unexpected response: {gpt_response}")
            result_gpt4.append("Check Manually")

    time.sleep(1)  # Delay to avoid hitting rate limits


#create a new row in dataset to store predicted values
dataset["gpt4_pred"] = result_gpt4

## Save the updated dataset

In [ ]:
# Save the updated dataset
dataset.to_excel("200_ARPs_and_Programming_Posts_with_predictions.xlsx", index=False)
print("Classification complete. Results saved.")


Classification complete. Results saved.


# Evaluation

In [ ]:
# ======== Evaluation ========
def scores(y_test, predictions):
    precision = metrics.precision_score(y_test, predictions)
    recall = metrics.recall_score(y_test, predictions)
    f1 = metrics.f1_score(y_test, predictions)
    acc = metrics.accuracy_score(y_test, predictions)


    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1}")
    print(f"Accuracy: {acc}")

# Ensure class labels exist in the dataset
if "Label" in dataset.columns:
    dataset = dataset.dropna(subset=["gpt4_pred"])  # Remove NaN values
    dataset["gpt4_pred"] = dataset["gpt4_pred"].astype(int)  # Convert to int

    scores(dataset["Label"], dataset["gpt4_pred"])
else:
    print("Error: Column 'class' not found in dataset. Check the correct column name for ground truth labels.")


F1-score: 0.875
Accuracy: 0.8805970149253731
Precision: 0.9130434782608695
Recall: 0.84
ROC AUC: 0.8803960396039604
